## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [1]:
from datetime import datetime
import re

class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self.codigo = codigo
        self.nombre = nombre
        self.tipo = tipo
        self._precio = precio
        self._cantidad = cantidad

    def valorTotal(self):
        return self._precio * self._cantidad

    @property
    def precio(self):
        return self._precio 

    @precio.setter
    def precio(self, precio):
        if 10 <= precio <= 2000:
            self._precio = precio

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, cantidad):
        if 0 <= cantidad <= 100:
            self._cantidad = cantidad

In [2]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [3]:
class Oferta:
    def __init__(self, descripcion="", codigos=[], tipos=[]):
        self.descripcion = descripcion
        self.codigos = codigos
        self.tipos = tipos

    @property
    def descripcion(self):
        return self._descripcion

    @descripcion.setter
    def descripcion(self, valor):
        if len(valor) <= 100:
            self._descripcion = valor

    def esAplicable(self, producto, cantidad):
        return producto.codigo in self.codigos or producto.tipo in self.tipos

    def aplicar(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            return producto.precio * cantidad - self.calcularDescuento(producto, cantidad)
        return producto.precio * cantidad


class OfertaDescuento(Oferta):
    def __init__(self, descuento, descripcion="", codigos=[], tipos=[]):
        super().__init__(descripcion, codigos, tipos)
        self.tipo = "descuento"
        self.descuento = descuento

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            return producto.precio * cantidad * (self.descuento / 100)
        return 0


class Oferta2x1(Oferta):
    def __init__(self, tipos, codigos=None):
        if codigos is None:
            codigos = []
        
        super().__init__(descripcion="Oferta 2x1", codigos=codigos, tipos=tipos)
        self.tipo = "2x1"

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            productos_gratis = cantidad // 2
            return productos_gratis * producto.precio
        return 0


In [4]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [5]:
class Catalogo:
    def __init__(self):
        self.productos = []
        self.ofertas = []

    @classmethod
    def leer(cls, archivo):
        import csv
        catalogo = cls()
        with open(archivo, 'r') as f:
            reader = csv.reader(f)
            next(reader) 
            for row in reader:
                codigo, nombre, precio, tipo, cantidad = row
                catalogo.agregar(Producto(codigo, nombre, float(precio), tipo, int(cantidad)))
        return catalogo

    def guardar(self, archivo):
        import csv
        with open(archivo, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['codigo', 'nombre', 'precio', 'tipo', 'cantidad'])
            for producto in self.productos:
                writer.writerow([producto.codigo, producto.nombre, producto.precio, producto.tipo, producto.cantidad])

    def agregar(self, *productos):
        self.productos.extend(productos)

    def buscar(self, codigo):
        for producto in self.productos:
            if producto.codigo == codigo:
                return producto
        return None

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def buscarOferta(self, producto, cantidad):
        mejor_oferta = None
        mayor_descuento = 0
        for oferta in self.ofertas:
            if oferta.esAplicable(producto, cantidad):
                descuento = oferta.calcularDescuento(producto, cantidad)
                if descuento > mayor_descuento:
                    mejor_oferta = oferta
                    mayor_descuento = descuento
        return mejor_oferta

    def calcularDescuento(self, producto, cantidad):
        oferta = self.buscarOferta(producto, cantidad)
        if oferta:
            return oferta.calcularDescuento(producto, cantidad)
        return 0

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(p.cantidad for p in self.productos)

    @property
    def valorTotal(self):
        return sum(p.valorTotal() for p in self.productos)

    def vender(self, producto, cantidad):
        producto.cantidad -= cantidad

    def informe(self):
        informe = "INFORME CATALOGO\n"
        informe += f"Cantidad de productos:   {self.cantidadProductos}\n"
        informe += f"Cantidad de unidades:    {self.cantidadUnidades}\n"
        precio_promedio = self.valorTotal / self.cantidadUnidades if self.cantidadUnidades > 0 else 0
        informe += f"Precio Promedio:       $ {precio_promedio:.2f}\n" 
        informe += f"Valor total:           $ {self.valorTotal:.2f}\n"
        informe += "Tipos de productos: \n"
        
        tipos = {}
        for p in self.productos:
            if p.tipo not in tipos:
                tipos[p.tipo] = {'unidades': 0, 'valor_total': 0}
            tipos[p.tipo]['unidades'] += p.cantidad
            tipos[p.tipo]['valor_total'] += p.valorTotal()
        
        for tipo, data in tipos.items():
            precio_promedio_tipo = data['valor_total'] / data['unidades'] if data['unidades'] > 0 else 0
            informe += f"  - {tipo:<18}: {data['unidades']:>4}u x $ {precio_promedio_tipo:.2f}\n"
        
        informe += "Ofertas:\n"
        for oferta in self.ofertas:
            if isinstance(oferta, Oferta2x1):
                precio_total_oferta = sum(p.precio for p in self.productos if oferta.esAplicable(p, p.cantidad))
                informe += f"  - Oferta 2x1: ${precio_total_oferta:.2f}\n"
                continue

            total_descuento = 0
            contador_productos = 0
            for p in self.productos:
                if oferta.esAplicable(p, p.cantidad):
                    descuento = oferta.calcularDescuento(p, p.cantidad)
                    total_descuento += descuento
                    if p.precio * p.cantidad > 0:
                        contador_productos += 1
            
            if contador_productos > 0:
                porcentaje_promedio = (total_descuento / sum(p.precio * p.cantidad for p in self.productos if oferta.esAplicable(p, p.cantidad))) * 100
                informe += f"  - {oferta.descripcion}: {int(porcentaje_promedio)}% de descuento ($ {total_descuento:.2f})\n"
            else:
                informe += f"  - {oferta.descripcion}: No se aplicaron descuentos\n"
        
        return informe

    def imprimir_informe(self):
        print(self.informe())


if __name__ == "__main__":
    catalogo = Catalogo.leer('catalogo.csv')

    o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
    od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
    od10 = OfertaDescuento(10, tipos=['fideo'])

    catalogo.registrarOferta(o2x1)
    catalogo.registrarOferta(od20)
    catalogo.registrarOferta(od10)
    catalogo.imprimir_informe()

INFORME CATALOGO
Cantidad de productos:   30
Cantidad de unidades:    1000
Precio Promedio:       $ 2000.00
Valor total:           $ 2000000.00
Tipos de productos: 
  - gaseosa           :  120u x $ 1399.17
  - galleta           :  255u x $ 2165.10
  - fideo             :  125u x $ 2096.00
  - yogur             :  150u x $ 2256.67
  - agua              :  150u x $ 2933.33
  - arroz             :  200u x $ 1197.50
Ofertas:
  - Oferta 2x1: $13900.00
  - : 20% de descuento ($ 81480.00)
  - : 10% de descuento ($ 26200.00)



In [6]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [7]:
class Cliente:
    def __init__(self, nombre_cliente, cuit_cliente):
        self._nombre = None
        self._cuit = None
        self.nombre = nombre_cliente
        self.cuit = cuit_cliente

    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, nuevo_nombre):
        if self.validar_nombre(nuevo_nombre):
            self._nombre = nuevo_nombre

    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, nuevo_cuit):
        if self.validar_cuit(nuevo_cuit):
            self._cuit = nuevo_cuit

    def validar_nombre(self, nombre):
        return 1 <= len(nombre) <= 100

    def validar_cuit(self, cuit):
        patron = r'^\d{2}-\d{8}-\d{1}$'
        return re.match(patron, cuit) is not None

In [8]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [9]:
class Factura:
    _ultimo_numero = 0

    @classmethod
    def ultimaFactura(cls, numero):
        cls._ultimo_numero = numero

    @classmethod
    def nuevoNumero(cls):
        cls._ultimo_numero += 1
        return cls._ultimo_numero

    def __init__(self, catalogo, cliente):
        self.numero = Factura.nuevoNumero()
        self.catalogo = catalogo
        self.cliente = cliente
        self.items = {}
        self.fecha = datetime.now().strftime('%Y-%m-%d')

    def agregar(self, producto, cantidad):
        self.items[producto] = self.items.get(producto, 0) + cantidad
        self.catalogo.vender(producto, cantidad)

    @property
    def cantidadProductos(self):
        return len(self.items)

    @property
    def cantidadUnidades(self):
        return sum(self.items.values())

    @property
    def subtotal(self):
        return sum(producto.precio * cantidad for producto, cantidad in self.items.items())

    @property
    def descuentos(self):
        return sum(self.catalogo.calcularDescuento(producto, cantidad) for producto, cantidad in self.items.items())

    @property
    def total(self):
        return self.subtotal - self.descuentos

    def imprimir(self):
        output = [
            f"Factura: {self.numero}",
            f"Fecha  : {self.fecha}",
            f"Cliente: {self.cliente.nombre} ({self.cliente.cuit})\n"
        ]

        for producto, cantidad in self.items.items():
            subtotal_producto = producto.precio * cantidad
            output.append(f"- {cantidad} {producto.nombre} x ${producto.precio:.2f} = ${subtotal_producto:.2f}")
            
            ofertas = {
                OfertaDescuento: lambda oferta: f"      Descuento {oferta.descuento}%{'':<30} - ${self.catalogo.calcularDescuento(producto, cantidad):.2f}",
                Oferta2x1: lambda oferta: f"      Oferta 2x1{'':<33} - ${self.catalogo.calcularDescuento(producto, cantidad):.2f}",
            }
            oferta = self.catalogo.buscarOferta(producto, cantidad)
            output.append(
                ofertas.get(type(oferta), lambda _: "")(oferta)
                if isinstance(oferta, (OfertaDescuento, Oferta2x1))
                else f"      {oferta.descripcion}{'':<41} - ${self.catalogo.calcularDescuento(producto, cantidad):.2f}"
            )

        output.append(f"\n{'Subtotal:':<50} ${self.subtotal:>8.2f}")
        output.append(f"{'Descuentos:':<50} ${self.descuentos:>8.2f}")
        output.append(f"{'-' * 70:>50}")
        output.append(f"{'TOTAL:':<50} ${self.total:>8.2f}")

        factura = "\n".join(output)
        print(factura)
        return factura


In [10]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Factura: 103
Fecha  : 2024-10-08
Cliente: Juan Perez (20-12345678-9)

- 10 Coca Cola x $1500.00 = $15000.00
      Descuento 10%                               - $1500.00
- 3 Sonrisa x $1200.00 = $3600.00
      Oferta 2x1                                  - $1200.00

Subtotal:                                          $18600.00
Descuentos:                                        $ 2700.00
----------------------------------------------------------------------
TOTAL:                                             $15900.00
Prueba pasada exitosamente!


In [11]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!
